In [1]:
!pip install soundfile
import os
from scipy.io import wavfile
import numpy as np
import scipy
import scipy.signal
import soundfile as sf
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import csv
from collections import namedtuple
import datetime
import time
from google.colab import drive
import h5py
import cv2
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split

# Connect to google drive



In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/')

Mounted at /content/drive


# Get the labels <br>
Low and highFrequencies are in col 5 and 6. <br>
File path is in col 7. <br>
Split the file to get the start time of the cur recording. <br>
Begin clock time is in col 10. <br>
Use the start and end clock time (in col 3,4) adding to the begin clock time to decide the time period. <br>
Sonotype is in col 11.


In [3]:
# all the sound files
labelDir = "Shared drives/Stethoscope for the rainforests/Training data set /"
labelFiles = ["SA_10Sep2020.txt", "Table_13AB_10Sep2020.txt", "Table_control_10Sep2020.txt"]

times = [] 
freqs = []
files = []
sonotypes = []
actTimes = []
groups = []
selection = []

for labelFile in labelFiles:
  print()
  labelFilePath = labelDir + labelFile
  with open(labelFilePath) as file:
    label_reader = csv.reader(file, delimiter='\t')
    for row in label_reader:
      # not use the first row
      if (not row[0].isnumeric()):
        continue

      # all the recording are either start at 0min or 30mins
      duration = float(row[4]) - float(row[3])
      timeWeight = [3600,60,1,0.0001]
      actSta = sum([a*b for a,b in zip(timeWeight, map(int,row[10].replace('.',":").split(':')))])
      actEnd = actSta + duration
      
      # get file name
      if labelFile == "SA_10Sep2020.txt":
        fileName = "Spatial analysis/" + row[7].split("Audios_SA\\")[-1].replace("\\","/")
      elif labelFile == "Table_13AB_10Sep2020.txt":
        fileName = "13AB/" + row[7].split("\\")[-1]
        # 3B_20180607_070500_Dawn_Sunrise_Based.wav
        # or 20180908_060000_13A_24H [1.4452 116.9811].wav
      elif labelFile == "Table_control_10Sep2020.txt":
        fileName = "Control sites/" + row[7].split("\\")[-1]
        
      splName = row[7].split("\\")[-1].split("_")

      # get the start time of cur recording
      # actual time is either in index 1 or 2 in splName
      if (len(splName[1]) == 6):
        strTime = splName[1]         
      elif (len(splName[2]) == 6):   
        strTime = splName[2]
      else: 
        print(row[0])
        continue  

      recLis = [int(strTime[0]) * 10 + int(strTime[1]),
                int(strTime[2]) * 10 + int(strTime[3]), 
                int(strTime[4]) * 10 + int(strTime[5])]    

      recSta = sum([a*b for a,b in zip(timeWeight, recLis)])

      # start, end time used to corp the specs
      start = actSta - recSta
      end = start + duration

      try:
        sonotypes.append(int(row[11]))
        times.append([start,end])
        actTimes.append([actSta, actEnd])
        freqs.append([float(row[5]), float(row[6])])
        files.append(fileName) 
        groups.append(row[12])
        selection.append(int(row[0]))
      except:
        # selection number
        print(filename)
        print(row[0])
        pass

print("dataset size:  %i" % len(times))    
print("filename sample: " + files[0])

# print(len(files))
print(actTimes[len(files)-1])
print(times[len(files)-1])




dataset size:  3783
filename sample: Spatial analysis/4B/20180607_6h_day_weekly/20180607_060500_Dawn_Sunrise__4B___.wav
[26616.9212, 26621.323112212023]
[1176.9212000000007, 1181.3231122120233]


In [4]:
# get the data for top k sonotypes
s_unique, s_freq = np.unique(sonotypes, return_counts=True)
s_freq_order = np.argsort(s_freq)[::-1]
s_freq_desc = s_freq[s_freq_order]

print(len(s_unique))
print(s_unique[s_freq_order][:121])
print(s_freq_desc[:121])

458
[ 52 138  25   1 283 463 318 236 167  86 175 234 220 139 110  90  -1  -3
 493 413  95  88 109 475 130  63 280  48  -4 367  56 207 460  61  -7 174
 219  82 321 131 481 427 431 193  77 113 102   7  87 136   2 532 119 176
 188 129 346 312 137 104  78 310 497 317  49 103 348 284 424 422  -6  91
 202  65 120 209 399 154 307 498 171  53 432 143  92 441 491 543 419 345
 336  97 197 517 208  45 385 247 340 198  42 372 420 494 179 524 544 466
 274 490 546 531 140 173 172 114 199  81 423 266 529]
[231 153 127 111 109  91  80  80  80  62  49  47  47  45  41  40  37  35
  35  33  33  32  32  32  31  30  29  29  28  26  26  25  25  25  24  24
  24  24  24  24  22  22  21  20  19  19  19  19  17  17  17  17  16  16
  15  15  15  15  15  15  15  15  15  15  14  14  14  14  14  13  13  13
  13  13  12  12  12  12  12  12  11  10  10  10  10  10  10  10   9   9
   9   9   9   9   9   9   8   8   8   8   8   8   8   8   8   8   8   8
   8   7   7   7   7   7   7   7   7   7   7   7   7]


# Creat H5 data storage


In [24]:
f.close()
f = h5py.File('My Drive/Stethoscope/whole_data_1110.hdf5', 'w')

data = f.create_dataset("specs", (0,224,224,3,), maxshape=(None,None,None,None,), chunks=True)
data = f.create_dataset("sonotypes", (0,),  maxshape=(None,), chunks=True)
data = f.create_dataset("times", (0,2,), maxshape=(None,None,), chunks=True)
data = f.create_dataset("freqs", (0,2,), maxshape=(None,None,), chunks=True)
data = f.create_dataset("groups", (0,),  maxshape=(None,), chunks=True, dtype="S10")
data = f.create_dataset("selections", (0,),  maxshape=(None,), chunks=True)

f.close()

# Get the data according to the labels and store into database


In [25]:
# all the sound files
fileDirPath = "Shared drives/Stethoscope for the rainforests/Sound data/EastKalimantan/"

usedFiles = np.unique(files)

for curFile in usedFiles:
  print("processing " + curFile)
  filePath = fileDirPath + curFile
  try:
    audio, rate = sf.read(filePath)
  except:
    print("ERROR READING FILE")
    continue

  # used to store info after resizing the image
  specs_resized = []
  freqs_updated = []
  times_updated = []
  sonotypes_updated = []
  groups_updated = []
  selections_updated = []

  # plot the spectrogram
  freq, t, spec = scipy.signal.spectrogram(audio, rate)

  for i in range(len(files)):
    if(files[i] == curFile):
      boxTime = times[i]
      boxFreq = freqs[i]

      # get low and high freq index of the box
      low_freq = np.argmin(np.abs(freq - boxFreq[0]))
      high_freq = np.argmin(np.abs(freq - boxFreq[1]))
      # get s tart and end time of the box
      start = np.argmin(np.abs(t - boxTime[0]))
      end = np.argmin(np.abs(t - boxTime[1]))

      # adjust the params to cover the whole box
      # be aware of index out of bound
      if freq[low_freq] > boxFreq[0]: 
        low_freq = max(low_freq -1,0)

      if freq[high_freq] < boxFreq[1]: 
        high_freq = high_freq + 1

      if t[start] > boxTime[0]: 
        start = max(start - 1, 0)

      if t[end] < boxTime[1]: 
        start = start + 1

      cur_spec = np.copy(spec[low_freq:high_freq, start:end])

      if cur_spec.size != 0:
        # # resize
        # spec_resized=cv2.resize(cur_spec.astype('float32'),(224,224))
        spec_resized=cv2.resize(cur_spec.astype('float32'),(28,28))
        spec_resized = np.expand_dims(spec_resized, 2)
        # spec_resized=cv2.cvtColor(spec_resized.astype('float32'), cv2.COLOR_BGR2RGB) #cv2 does not accept float64 
        spec_resized = np.flip(spec_resized,0)

        specs_resized.append(spec_resized)
        freqs_updated.append(freqs[i])
        times_updated.append(actTimes[i])
        sonotypes_updated.append(sonotypes[i])
        groups_updated.append(groups[i]) 
        selections_updated.append(selection[i]) 
  
  # store to H5 data storage once after parsing each audio file
  # to avoid mem crash
  if len(specs_resized) != 0:

    f =  h5py.File('My Drive/Stethoscope/whole_data_1110.hdf5', "a")
    print(f["specs"].shape)
    
    f["times"].resize((f["times"].shape[0] + len(times_updated)), axis = 0)
    f["times"][-len(times_updated):,:] = times_updated

    f["freqs"].resize((f["freqs"].shape[0] +  len(times_updated)), axis = 0)
    f["freqs"][-len(times_updated):,:] = freqs_updated

    f["specs"].resize((f["specs"].shape[0] +  len(times_updated)), axis = 0)
    f["specs"][-len(times_updated):,:,:,:] = specs_resized

    f["sonotypes"].resize((f["sonotypes"].shape[0] + len(times_updated)), axis = 0)
    f["sonotypes"][-len(times_updated):] = np.array(sonotypes_updated)

    f["groups"].resize((f["groups"].shape[0] + len(times_updated)), axis = 0)
    f["groups"][-len(times_updated):] = np.array(groups_updated, dtype="S")

    f["selections"].resize((f["selections"].shape[0] + len(times_updated)), axis = 0)
    f["selections"][-len(times_updated):] = np.array(selections_updated)

    f.close()
  else:
    print(curFile)


processing 13AB/20180607_060400_Dawn_Sunrise____13B_.4455 116.9813].wav
(0, 28, 28, 1)
processing 13AB/20180607_120400_Midday_Sunrise___13B_.4455 116.9813].wav
(13, 28, 28, 1)
processing 13AB/20180607_174600_Dusk___13B_ [1.4455 116.9813].wav
(15, 28, 28, 1)
processing 13AB/20180710_060000_13A_24H [1.4452 116.9811].wav
(30, 28, 28, 1)
processing 13AB/20180710_093000_13A_24H [1.4452 116.9811].wav
(101, 28, 28, 1)
processing 13AB/20180710_100000_13A_24H [1.4452 116.9811].wav
(114, 28, 28, 1)
processing 13AB/20180710_180000_13A_24H [1.4452 116.9811].wav
(115, 28, 28, 1)
processing 13AB/20180809_060000_13A_24H [1.4452 116.9811].wav
(150, 28, 28, 1)
processing 13AB/20180809_120000_13A_24H [1.4452 116.9811].wav
(196, 28, 28, 1)
processing 13AB/20180809_180000_13A_24H [1.4452 116.9811].wav
(205, 28, 28, 1)
processing 13AB/20180908_060000_13A_24H [1.4452 116.9811].wav
(362, 28, 28, 1)
processing 13AB/20180908_103000_13A_24H [1.4452 116.9811].wav
(443, 28, 28, 1)
processing 13AB/20180908_110000_